In [1]:
from pyspark.sql import SparkSession

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Products and Categories") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/06 01:56:42 WARN Utils: Your hostname, MacBook-Pro-Temirlan.local, resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
25/09/06 01:56:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/06 01:56:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
products = [
    (1, "Laptop"),
    (2, "Phone"),
    (3, "Tablet"),
    (4, "Headphones")
]

products_df = spark.createDataFrame(products, ["product_id", "product_name"])
products_df.show()

+----------+------------+
|product_id|product_name|
+----------+------------+
|         1|      Laptop|
|         2|       Phone|
|         3|      Tablet|
|         4|  Headphones|
+----------+------------+



In [3]:
categories = [
    (1, "Electronics"),
    (2, "Gadgets"),
    (3, "Audio")
]

categories_df = spark.createDataFrame(categories, ["category_id", "category_name"])
categories_df.show()

+-----------+-------------+
|category_id|category_name|
+-----------+-------------+
|          1|  Electronics|
|          2|      Gadgets|
|          3|        Audio|
+-----------+-------------+



In [4]:
product_category = [
    (1, 1),  # Laptop → Electronics
    (2, 1),  # Phone → Electronics
    (2, 2),  # Phone → Gadgets
    (4, 3)   # Headphones → Audio
]

product_category_df = spark.createDataFrame(product_category, ["product_id", "category_id"])
product_category_df.show()

+----------+-----------+
|product_id|category_id|
+----------+-----------+
|         1|          1|
|         2|          1|
|         2|          2|
|         4|          3|
+----------+-----------+



In [5]:
result = products_df.join(
    product_category_df,
    products_df.product_id == product_category_df.product_id,
    "left"
).join(
    categories_df,
    product_category_df.category_id == categories_df.category_id,
    "left"
).select(
    products_df.product_name,
    categories_df.category_name
)

result.show()

+------------+-------------+
|product_name|category_name|
+------------+-------------+
|      Laptop|  Electronics|
|       Phone|      Gadgets|
|       Phone|  Electronics|
|      Tablet|         NULL|
|  Headphones|        Audio|
+------------+-------------+



In [6]:
no_category = result.filter(result.category_name.isNull()).select("product_name")
no_category.show()

+------------+
|product_name|
+------------+
|      Tablet|
+------------+



In [7]:
pairs = get_product_category_pairs(products, categories, product_category)
pairs.show()

no_categories = pairs.filter("category IS NULL").select("product")
no_categories.show()

NameError: name 'get_product_category_pairs' is not defined

In [8]:
pairs = get_product_category_pairs(products, categories, product_category)
pairs.show()

no_categories = pairs.filter("category IS NULL").select("product")
no_categories.show()

NameError: name 'get_product_category_pairs' is not defined

In [9]:
from pyspark.sql import DataFrame

def get_product_category_pairs(products: DataFrame, categories: DataFrame, product_category: DataFrame) -> DataFrame:
    return products.join(
        product_category, products.id == product_category.product_id, "left"
    ).join(
        categories, product_category.category_id == categories.id, "left"
    ).select(
        products.name.alias("product"), categories.name.alias("category")
    )

In [10]:
pairs = get_product_category_pairs(products, categories, product_category)
pairs.show()

no_categories = pairs.filter("category IS NULL").select("product")
no_categories.show()

AttributeError: 'list' object has no attribute 'join'

In [11]:
from pyspark.sql import Row

products = spark.createDataFrame([
    Row(id=1, name="Яблоко"),
    Row(id=2, name="Банан"),
    Row(id=3, name="Апельсин"),
    Row(id=4, name="Арбуз"),
])

categories = spark.createDataFrame([
    Row(id=1, name="Фрукты"),
    Row(id=2, name="Цитрусовые"),
])

product_category = spark.createDataFrame([
    Row(product_id=1, category_id=1),  # Яблоко → Фрукты
    Row(product_id=2, category_id=1),  # Банан → Фрукты
    Row(product_id=3, category_id=2),  # Апельсин → Цитрусовые
])

In [12]:
from pyspark.sql import DataFrame

def get_product_category_pairs(products: DataFrame, categories: DataFrame, product_category: DataFrame) -> DataFrame:
    return products.join(
        product_category, products.id == product_category.product_id, "left"
    ).join(
        categories, product_category.category_id == categories.id, "left"
    ).select(
        products.name.alias("product"), categories.name.alias("category")
    )

In [13]:
pairs = get_product_category_pairs(products, categories, product_category)
pairs.show()

no_categories = pairs.filter("category IS NULL").select("product")
no_categories.show()

+--------+----------+
| product|  category|
+--------+----------+
|  Яблоко|    Фрукты|
|   Банан|    Фрукты|
|Апельсин|Цитрусовые|
|   Арбуз|      NULL|
+--------+----------+

+-------+
|product|
+-------+
|  Арбуз|
+-------+

